In [2]:
import os
import sys
import pandas as pd
import logging
from google.cloud import bigquery
from hashlib import md5
from typing import List
import uuid
from pandas.io.json import json_normalize
import pycountry

In [46]:
#creating dim_country
df = pd.read_csv('./data/Emissions_by_Country_2002-2022.csv')
lat_long_df = pd.read_csv('./data/world_country_and_usa_states_latitude_and_longitude_values.csv')
df1 = df[['Country', 'ISO 3166-1 alpha-3']]
#make a 2 letter country code
def alpha3code(column):
    CODE=[]
    for country in column:
        try:
            code=pycountry.countries.get(name=country).alpha_3
           # .alpha_3 means 3-letter country code 
           # .alpha_2 means 2-letter country code
            CODE.append(code)
        except:
            CODE.append('None')
    return CODE
#make a 3 digit country code column in my lat_long_df
lat_long_df['long_code']=alpha3code(lat_long_df.country)
lat_long_df = lat_long_df[['country','country_code','long_code','latitude','longitude']]
#read country size csv
size_df = pd.read_csv('./data/country_size.csv', dtype=str)
size_df = size_df[['Country Code','2020']]
size_df["2020"] = size_df['2020'].astype('float').round(decimals=0)
size_df = size_df.rename(columns={'2020': 'land_area_square_km', 'Country Code': 'long_code'})

#merge datasets to create dim_countries
dim_country = pd.merge(lat_long_df,size_df, how='inner', on='long_code')
dim_country["created_at"] = dim_country.apply(lambda _: ' ', axis=1)
dim_country["modified_at"] = dim_country.apply(lambda _: ' ', axis=1)
display(dim_country)

,country,country_code,long_code,latitude,longitude,land_area_square_km,created_at,modified_at
0,Andorra,AD,AND,42.546245,1.601554,470.0,,
1,United Arab Emirates,AE,ARE,23.424076,53.847818,71020.0,,
2,Afghanistan,AF,AFG,33.939110,67.709953,652230.0,,
3,Antigua and Barbuda,AG,ATG,17.060816,-61.796428,440.0,,
4,Albania,AL,ALB,41.153332,20.168331,27400.0,,
...,...,...,...,...,...,...,...,...
181,Samoa,WS,WSM,-13.759029,-172.104629,2780.0,,
182,Yemen,YE,YEM,15.552727,48.516388,527970.0,,
183,South Africa,ZA,ZAF,-30.559482,22.937506,1213090.0,,
184,Zambia,ZM,ZMB,-13.133897,27.849332,743390.0,,


In [43]:
#prepare languages spoken data set
language_df = pd.read_csv('./data/countries-languages.csv')
display(language_df)

,Country,Languages Spoken
0,Afghanistan,"Dari Persian, Pashtu (both official), other Tu..."
1,Albania,"Albanian (Tosk is the official dialect), Greek"
2,Algeria,"Arabic (official), French, Berber dialects"
3,Andorra,"Catalán (official), French, Castilian, Portuguese"
4,Angola,"Portuguese (official), Bantu and other African..."
...,...,...
193,Vietnam,Vietnamese (official); English (increasingly f...
194,Western Sahara (proposed state),"Hassaniya Arabic, Moroccan Arabic"
195,Yemen,Arabic
196,Zambia,"English (official); major vernaculars: Bemba, ..."
